# Assignment

## Connecting to Database

In [ ]:
from __future__ import (absolute_import, division, print_function)
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import os
import json
import requests


import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point
from shapely import wkt
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
from pprint import pprint

def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = ''
    YOUR_PW     = ''
    DB_LOGIN    = 'y20s1d2x01_'+ YOUR_UNIKEY

    try:
        db_conn_dict = {'host': 'soitpw11d59.shared.sydney.edu.au', 'port': 5432, 'database': DB_LOGIN, 'user':DB_LOGIN , 'password': YOUR_PW }
        conn = psycopg2.connect(**db_conn_dict)
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
        return None
    return conn

def pgquery( conn, sqlcmd, args=None, msg=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholders in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if msg != False:
                    print("success: " + msg)
            except psycopg2.DatabaseError as e:
                if e.pgcode != None:
                    if msg: print("db read error: "+msg)
                    print(e)
            except Exception as e:
                print(e)
    return retval

In [ ]:
# Connecting to database
conn = pgconnect()

## Task 1: Data Integration and Database Generation

### Creating Tables

In [ ]:
statistical_areas_table = """CREATE TABLE statistical_areas (
                area_id INTEGER PRIMARY KEY,
                area_name VARCHAR(50) , 
                parent_area_id INTEGER
            );"""

neighbourhoods_table = """CREATE TABLE neighbourhoods (
                area_id INTEGER NOT NULL,
                area_name VARCHAR(50),
                land_area NUMERIC,
                population INTEGER,
                number_of_dwellings INTEGER,
                number_of_businesses INTEGER,
                median_annual_household_income INTEGER,
                avg_monthly_rent INTEGER,
                PRIMARY KEY (area_id),
                FOREIGN KEY (area_id) REFERENCES statistical_areas(area_id)
            );"""

census_stats_table = """CREATE TABLE census_stats (
                area_id INTEGER PRIMARY KEY, 
                median_annual_household_income INTEGER,
                avg_monthly_rent INTEGER
            );"""

population_stats_2016_table = """CREATE TABLE population_stats_2016 (
                area_id INTEGER PRIMARY KEY,
                area_name VARCHAR(50),
                age_0_to_4 INTEGER,
                age_5_to_9 INTEGER,
                age_10_to_14 INTEGER,
                age_15_to_19 INTEGER,
                age_20_to_24 INTEGER,
                age_25_to_29 INTEGER,
                age_30_to_34 INTEGER,
                age_35_to_39 INTEGER,
                age_40_to_44 INTEGER,
                age_45_to_49 INTEGER,
                age_50_to_54 INTEGER,
                age_55_to_59 INTEGER,
                age_60_to_64 INTEGER,
                age_65_to_69 INTEGER,
                age_70_to_74 INTEGER,
                age_75_to_79 INTEGER,
                age_80_to_84 INTEGER,
                age_85_and_over INTEGER,
                total_persons INTEGER,
                females INTEGER,
                males INTEGER
            );"""

health_services_table = """CREATE TABLE health_services (
                id INTEGER PRIMARY KEY,
                name VARCHAR(100),
                category VARCHAR(50),
                num_beds INTEGER,
                address VARCHAR(100),
                suburb VARCHAR(50),
                state VARCHAR(4),
                postcode CHAR(4),
                longitude NUMERIC,
                latitude NUMERIC,
                comment VARCHAR(2300),
                website VARCHAR(150)
            );"""

nsw_postcodes_table = """CREATE TABLE nsw_postcodes (
                id INTEGER PRIMARY KEY,
                postcode INTEGER,
                locality VARCHAR(50),
                longitude NUMERIC,
                latitude NUMERIC
            );"""

nsw_testsites_table = """CREATE TABLE nsw_testsites (
                site_id INTEGER PRIMARY KEY,
                Centre_name VARCHAR(70) ,
                Phone_number VARCHAR(70),
                Opening_hours VARCHAR(80),
                longitude NUMERIC,
                latitude NUMERIC,
                test_capacity INTEGER
            );"""

covid_tests_table = """CREATE TABLE covid_tests (
                test_id INTEGER PRIMARY KEY,
                test_date DATE,
                postcode CHAR(4),
                lhd_2010_code CHAR(4),
                lhd_2010_name VARCHAR(50),
                lga_code19 INTEGER,
                lga_name19 VARCHAR(50),
                result VARCHAR(50)
            );"""

SA2_data_table = """CREATE TABLE sa2_data (
                id INTEGER PRIMARY KEY,
                name VARCHAR(50),
                geom GEOMETRY(MULTIPOLYGON,4326)
            );"""

# DO NOT EXECUTE THIS CODE - the tables have already been created in the database
pgquery(conn, "SET search_path to covid")
pgquery(conn, statistical_areas_table)
pgquery(conn, neighbourhoods_table)
pgquery(conn, census_stats_table)
pgquery(conn, population_stats_2016_table)
pgquery(conn, health_services_table)
pgquery(conn, nsw_testsites_table)
pgquery(conn, covid_tests_table)

# We need a reference to the public schema in order to use PostGIS
pgquery(conn, "SET search_path to covid, public")
pgquery(conn, SA2_data_table)

### Loading data and inserting them into tables

In [ ]:
# DO NOT EXECUTE THIS CODE - the data has already been loaded into the database

conn.execute("SET search_path to covid")


# Loading data into statistical_areas
statistical_areas_data = pd.read_csv("datasets/StatisticalAreas.csv")
for index, row in statistical_areas_data.iterrows():
    conn.execute("INSERT INTO statistical_areas VALUES (%s, %s, %s)", (row[0], row[1], row[2]))


# Loading data into neighbourhoods
neighbourhoods_data = pd.read_csv("datasets/Neighbourhoods.csv")
for index, row in neighbourhoods_data.iterrows():
    # sorry about this
    if str(row[2]) == "nan":
        row[2] = None
    if str(row[3]) == "nan":
        row[3] = None
    if str(row[4]) == "nan":
        row[4] = None
    if str(row[5]) == "nan":
        row[5] = None
    if str(row[6]) == "nan":
        row[6] = None
    if str(row[7]) == "nan":
        row[7] = None
    conn.execute("INSERT INTO neighbourhoods VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]))




# Loading data into nsw_testsites
covid19_nsw_testsites_simulated_capacity_data = pd.read_csv("datasets/covid19_nsw_testsites_simulated_capacity.csv")
for index, row in covid19_nsw_testsites_simulated_capacity_data.iterrows():
    conn.execute("INSERT INTO nsw_testsites VALUES (%s, %s, %s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4], row[5], row[6]))

# Loading data into nsw_postcodes
nsw_postcodes_data = pd.read_csv("datasets/NSW_Postcodes.csv")
for index, row in nsw_postcodes_data.iterrows():
    conn.execute("INSERT INTO nsw_postcodes VALUES (%s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4]))

# Loading data into population_stats_2016
population_stats_2016_data = pd.read_csv("datasets/PopulationStats2016.csv")
for index, row in population_stats_2016_data.iterrows():
    # oh my god
    row_tuple = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19], row[20], row[21], row[22])
    conn.execute("INSERT INTO population_stats_2016 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", row_tuple)


# Loading data into census_stats
census_stats_data = pd.read_csv("datasets/CensusStats.csv")
for index, row in census_stats_data.iterrows():
    # Numpy data types don't like being converted to None
    if str(row[1]) == "nan" and str(row[2]) != "nan":
        conn.execute("INSERT INTO census_stats VALUES (%s, %s, %s)", (row[0], None, row[2]))
        continue
    if str(row[2]) == "nan" and str(row[1]) != "nan":
        conn.execute("INSERT INTO census_stats VALUES (%s, %s, %s)", (row[0], row[1], None))
        continue
    if str(row[2]) == "nan" and str(row[1]) == "nan":
        conn.execute("INSERT INTO census_stats VALUES (%s, %s, %s)", (row[0], None, None))
        continue
    conn.execute("INSERT INTO census_stats VALUES (%s, %s, %s)", (row[0], row[1], row[2]))

# Loading data into health_services
health_services_data = pd.read_csv("datasets/HealthServices.csv")
for index, row in health_services_data.iterrows():
    if str(row[3]) == "nan":
        row[3] = None
    conn.execute("INSERT INTO health_services VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10]))

# Loading data into covid_tests
with open("datasets/tests_by_date_location_result.json", "r") as f:
    covid_test_data = json.load(f)
for record in covid_test_data['records']:
    conn.execute("INSERT INTO covid_tests VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (record[0], record[1], record[2], record[3], record[4], record[5], record[6], record[7]))

# Loading spatial data in sa2
geom_wkt_data = []
for index, row in SA2_data.iterrows():
    try:
        geom_wkt_data.append(row['geometry'].wkt) # converting geometry polygon data to a string format
    except:
        geom_wkt_data.append(None)
SA2_data = SA2_data.assign(geom_wkt = geom_wkt_data) # adding string formatted geometry to dataframe
conn.execute("SET search_path to covid, public")
for index, row in SA2_data.iterrows():
    conn.execute("INSERT INTO sa2_data VALUES (%s, %s, ST_Multi(ST_GeomFromText(%s, 4326)))", (row[0], row[2], row[13]))

## Add in geometry columns for health_services, NSW_Postcodes & Neighbourhoods

In [ ]:
pgquery(conn, "SET search_path to covid, public")

# Adding geom column to neighbourhoods
pgquery(conn, "ALTER TABLE health_services ADD geom GEOMETRY(POINT, 4326)")
for i in range(3717):
    query = """UPDATE health_services SET geom = 
        (SELECT ST_SetSRID(ST_Point(CAST(longitude as float), CAST(latitude as float)), 4326) FROM health_services WHERE id = {})
        WHERE id = {}
        """.format(i, i)
    pgquery(conn, query)
    
# Adding geom column to postcodes    
pgquery(conn, "ALTER TABLE nsw_postcodes ADD geom GEOMETRY(POINT, 4326)")
for i in range(22873):
    query = """UPDATE nsw_postcodes SET geom = 
        (SELECT ST_SetSRID(ST_Point(CAST(longitude as float), CAST(latitude as float)), 4326) FROM nsw_postcodes WHERE id = {})
        WHERE id = {}
        """.format(i, i)
    pgquery(conn, query)

# Adding geom column to neighbourhoods
pgquery(conn, "ALTER TABLE neighbourhoods ADD geom GEOMETRY(MULTIPOLYGON, 4326)")
res = pgquery(conn, "SELECT area_id FROM neighbourhoods")
for index in res:
    query = """UPDATE neighbourhoods SET geom = 
         (SELECT geom FROM sa2_data WHERE id = {})
         WHERE area_id = {}
         """.format(index[0], index[0])
    pgquery(conn, query)

### UPDATE to LATEST JSON covid test cases !!!!

In [ ]:
# # Get the JSON 
json_url="https://data.nsw.gov.au/data/datastore/dump/227f6b65-025c-482c-9f22-a25cf1b8594f?format=json"
response = requests.get(json_url)


In [ ]:
covid_test_data = response.json()
covid_test_data
# Delete everything in the covid_test_data table
pgquery(conn, "DELETE FROM covid_tests")

# Insert into the database. Note: it will take 5 minutes.
for record in covid_test_data['records']:
    insert = "INSERT INTO covid_tests VALUES ({}, '{}', '{}', '{}', '{}', {}, '{}', '{}')".format(record[0], record[1], record[2], record[3], record[4], record[5], record[6], record[7]) 
    print(pgquery(conn, insert))
# # print(insert)

## Spatial Joins on neighbourhoods and health_services
### Tips for searching:
* Before any queries, please connect to database, UPDATE TO LATEST covid test cases and then run pgquery()


* For querying using any postgis methods: 
    1. pgquery(conn, "set search_path to covid,public;") 
    2. Do your query (e.g. pgquery(conn, "select ST_AsText(geom) from sa2_data limit 2;")


* Use ST_AsText() postgis method to check if geometry type of data in geom column


* When using gpd.read_postgis, remember you MUST ALWAYS specify a geom column in query!


In [ ]:
pgquery(conn, "set search_path to covid,public;")

### Health services spatial index to increase query speed


In [ ]:
# index_command = "CREATE INDEX health_services_idx ON health_services USING GIST (geom)"
# result= pgquery(conn, index_command, returntype='dict')
# pprint(result)

## Find health service density per neighbourhood
* We did a spatial ST_Within join between Neighbourhoods and Health Services, thereby finding the health service density per neighbourhood, where neighbourhood is the neighbourhood population/1000.

In [ ]:
query = """SELECT n.area_name, n.land_area, n.population, count(h.name) AS number_health_services,
			 count(h.name)/(n.population::float/1000) AS "healthservice_density", n.geom
             FROM neighbourhoods n JOIN health_services h ON (ST_Within(h.geom, n.geom))
             GROUP BY n.area_name, n.land_area, n.population, n.geom
             ORDER BY n.area_name ASC;
            """

healthservice_density = gpd.read_postgis(query, conn)
healthservice_density

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.set_title("Health Services Density", fontsize=10.0)
legend_kwds = {'label': "Health Services Density", 'orientation': "horizontal"}
healthservice_density.plot(column="healthservice_density", figsize=(10, 10), cmap="tab20", ax=ax, legend=True, legend_kwds=legend_kwds)

In [ ]:
hsd_col = healthservice_density["healthservice_density"].to_numpy()
plt.hist(hsd_col, color="r", edgecolor="grey")
plt.xlabel("Health Service Density")
plt.ylabel("Frequency")
plt.title("Health Service Density per Neighbourhood")
plt.show()

## Find Population density per neighbourhood land area

In [ ]:
query = """select area_name, population/land_area AS population_density, geom from neighbourhoods 
            ORDER BY population_density DESC;
            """

population_density = gpd.read_postgis(query,conn)
population_density

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.set_title("Population Density", fontsize=10.0)
legend_kwds = {'label': "Population Density", 'orientation': "horizontal"}
population_density.plot(column="population_density", figsize=(10, 10), cmap="tab20", ax=ax, legend=True, legend_kwds=legend_kwds)

In [ ]:
popd_col = population_density["population_density"].to_numpy()
plt.hist(popd_col, color="r", edgecolor="grey")
plt.xlabel("Population Density")
plt.ylabel("Frequency")
plt.title("Population Density per Neighbourhood")
plt.show()

## Find Population Age Percentage

In [ ]:
query = """select area_name,CASE
WHEN total_persons = NULL OR total_persons = 0 Then NULL
ELSE 100*(age_70_to_74 + age_75_to_79 + age_80_to_84 + age_85_and_over)/total_persons::float 
END AS "Percentage_of_population_age_70_over", geom
from population_stats_2016 p LEFT JOIN sa2_data s ON (p.area_id = s.id);"""

population_age = gpd.read_postgis(query,conn)
population_age

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.set_title("Percentage of Population over 70", fontsize=10.0)
legend_kwds = {'label': "Percentage of Population over 70", 'orientation': "horizontal"}
population_age.plot(column="Percentage_of_population_age_70_over", figsize=(10, 10), cmap="tab20", ax=ax, legend=True, legend_kwds=legend_kwds)

In [ ]:
perc_pop_age = population_age["Percentage_of_population_age_70_over"].to_numpy()
plt.hist(perc_pop_age, color="r", edgecolor="grey")
plt.xlabel("Percentage of population aged 70 or over")
plt.ylabel("Frequency")
plt.title("Percentage of population aged 70 or over per neighbourhood")
plt.show()

## Find hospital bed density

In [ ]:
query = """SELECT n.area_name, SUM(h.num_beds)/(n.population/1000) AS "num_beds_density", n.geom
             FROM neighbourhoods n JOIN health_services h ON (ST_Within(h.geom, n.geom)) 
			 GROUP BY n.area_name,n.population,n.geom
			 ORDER BY num_beds_density DESC NULLS LAST;"""

hospital_bed_density = gpd.read_postgis(query,conn)
hospital_bed_density


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.set_title("Hospital Bed Density", fontsize=10.0)
legend_kwds = {'label': "Hospital Bed Density", 'orientation': "horizontal"}
hospital_bed_density.plot(column="num_beds_density", figsize=(10, 10), cmap="tab20", ax=ax, legend=True, legend_kwds=legend_kwds)

In [ ]:
num_beds_density = hospital_bed_density["num_beds_density"].to_numpy()
plt.hist(num_beds_density, color="r", edgecolor="grey")
plt.xlabel("Number of beds density")
plt.ylabel("Frequency")
plt.title("Number of beds density per neighbourhood")
plt.show()

### Indexing on sa2_data and nsw_postcodes
* Upon attempting a join between sa2_data and nsw_postcodes, we realised that the query was extremely slow, hence we created 2 indexes: one on sa2_data's geom column and the other on nsw_postcodes's geom column. This dropped the waiting time from 1 minute and 3 seconds to a mere 2 seconds.

In [ ]:
# index_command = "CREATE INDEX sa2_data_idx ON sa2_data USING GIST (geom)"
# result= pgquery(conn, index_command, returntype='dict')
# pprint(result)

# index_command = "CREATE INDEX nsw_postcodes_idx ON nsw_postcodes USING GIST (geom)"
# result= pgquery(conn, index_command, returntype='dict')
# pprint(result)


## Find Covid tests per neighbourhood

In [ ]:
query = """select neigh.area_name, count(ct.result) AS total_number_of_tests, neigh.geom FROM covid_tests ct 
JOIN nsw_postcodes np ON (ct.postcode = np.postcode) 
JOIN neighbourhoods neigh ON (ST_WITHIN(np.geom,neigh.geom))
GROUP BY neigh.area_name,neigh.geom 
ORDER BY total_number_of_tests DESC;"""

covid_tests = gpd.read_postgis(query,conn)
covid_tests

# Creating joint dataframe of the values

In [ ]:
new_df_1 = pd.DataFrame(population_density, columns = ['area_name', 'population_density'])
new_df_2 = pd.DataFrame(population_age, columns = ['area_name', 'Percentage_of_population_age_70_over'])
new_df_3 = pd.DataFrame(healthservice_density, columns = ['area_name',  'healthservice_density'])
new_df_4 = pd.DataFrame(hospital_bed_density, columns = ['area_name', 'num_beds_density'])
new_df_5 = pd.DataFrame(covid_tests, columns = ['area_name',  'total_number_of_tests','geom'])
#df_final = pd.concat([new_df_1, new_df_2, new_df_3, new_df_4])

df_final = pd.merge(new_df_1, new_df_2, on='area_name')
df_final = pd.merge(df_final, new_df_3, on='area_name')
df_final = pd.merge(df_final, new_df_4, on='area_name')
df_final = pd.merge(df_final, new_df_5, on='area_name')


# Calculate averages and standard deviations
avg_series_across_columns = df_final.mean(axis=0)
std_series_across_columns = df_final.std(axis=0,ddof=0)

def z_score(cur_value, column):
    return (cur_value - avg_series_across_columns[column])/std_series_across_columns[column]

new_column = []


for index, cur_row in df_final.iterrows():
    total_z_scores = (z_score(cur_row['population_density'], 'population_density') + z_score(
        cur_row['Percentage_of_population_age_70_over'], 'Percentage_of_population_age_70_over')
        + z_score(cur_row['healthservice_density'], 'healthservice_density')
        + z_score(cur_row['num_beds_density'], 'num_beds_density')
        + z_score(cur_row['total_number_of_tests'], 'total_number_of_tests'))
#     new_column.append(cur_row['population_density'])
    print(total_z_scores)

# df_final['Vulnerability'] = new_column
# df_final

#final_data_table = """CREATE TABLE final_data (
 #               area_name VARCHAR(50),
 #               num_beds_density INTEGER,
 #               population_age INTEGER,
  #              population_density INTEGER,
  #              healthservice_density INTEGER
  #              
  #          );"""

# pgquery(conn, "SET search_path to covid")
# pgquery(conn, final_data_table)

# # Loading data into final_data_table

# for index, row in df_final.iterrows():
#     if str(row[1]) == "nan":
#         row[1] = None
#     if str(row[2]) == "nan":
#         row[2] = None
#     if str(row[3]) == "nan":
#         row[3] = None
#     if str(row[4]) == "nan":
#         row[4] = None
#     conn.execute("INSERT INTO final_data VALUES (%s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4]))

## Closing Connection

In [ ]:
conn.close()